In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from datasets import load_dataset
import tqdm
from transformers import GPT2TokenizerFast, BertTokenizerFast, Trainer, TrainingArguments, EvalPrediction
import re
import tiktoken
import numpy as np
from bus_nGPT import Decoder, TransformerLayer, AttentionHead, Rotary, LMHead
import tensorboard as tb
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from importlib import reload

2024-12-02 20:09:04.072804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733191744.090759 1016245 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733191744.096366 1016245 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 20:09:04.116242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
baseline = torch.load('hyperscale.pt')

/tmp/ipykernel_1016245/4237985969.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline = torch.load('hyperscale.pt')


In [3]:
snli = load_dataset('stanfordnlp/snli')
mnli = load_dataset('nyu-mll/multi_nli')

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-uncased')

In [5]:
def tokenize_fn(examples):
    return tokenizer(examples['premise'] + " " + examples['hypothesis'], 
                     padding='max_length', 
                     truncation=True,
                     )

In [6]:
def compute_accuracy(eval: EvalPrediction):
    x = np.argmax(eval.predictions[0], axis=-1)
    z = np.average(x == eval.predictions[1])
    return {
        'accuracy': z    
        }

In [7]:
def abs_func(example):
    return {'label': example['label']} if example['label'] >= 0 else {'label':-example['label']}

In [8]:
snli = snli.map(abs_func)

In [9]:
snli_tokenized = snli.map(tokenize_fn)
# mnli_tokenized = mnli.map(tokenize_fn)

In [17]:
bl = LMHead(baseline, 512, 3)
bl.to('cuda')

LMHead(
  (ffn): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=3, bias=True)
  )
  (model): Decoder(
    (blocks): ModuleList(
      (0-7): 8 x TransformerLayer(
        (heads): ModuleList(
          (0-7): 8 x AttentionHead(
            (rope): Rotary()
          )
        )
        (silu): SiLU()
      )
    )
    (embeddings): Embedding(30523, 256)
  )
)

In [11]:
snli_tokenized

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 550152
    })
})

In [18]:

training_args = TrainingArguments(
	output_dir="output", 
	eval_strategy="steps", 
	num_train_epochs = 2,
	warmup_steps = 0,
	logging_steps = 100,
	save_steps = 100,
	load_best_model_at_end = True,
	learning_rate = 1e-3,
    per_device_train_batch_size=8,
    label_names=['labels'],
    gradient_accumulation_steps=200,
	)

trainer = Trainer(
    model=bl,
    args=training_args,
    train_dataset=snli_tokenized['train'],
    eval_dataset=snli_tokenized['validation'],
	tokenizer=tokenizer,
    compute_metrics=compute_accuracy

)



/tmp/ipykernel_1016245/2969620112.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

  0%|          | 0/686 [00:00<?, ?it/s]

In [15]:
trainer.evaluate(snli_tokenized['validation'])

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.079481840133667,
 'eval_accuracy': 0.4095,
 'eval_runtime': 61.3482,
 'eval_samples_per_second': 163.004,
 'eval_steps_per_second': 20.376,
 'epoch': 1.9953321991013393}

In [14]:
trainer.evaluate(snli_tokenized['test'])

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.079468011856079,
 'eval_accuracy': 0.4098,
 'eval_runtime': 61.0158,
 'eval_samples_per_second': 163.892,
 'eval_steps_per_second': 20.487,
 'epoch': 1.9953321991013393}